In [3]:
import os
import pandas as pd
import numpy as np
import sys
import traceback

#my libraries
src_path = 'path_to_src'
datar = 'path_to_data'
# add src_path if it's located in another path
#sys.path.append(os.path.join(src_path,'multimodal_retrieval/'))

In [2]:
# General function to compute average precision for retrieval given other samples as inputs
def get_AP(sorted_scores, given_sample, top_k, GT_samples):
        consider_top = sorted_scores[:top_k]
        top_sample_classes = [GT_samples[i[0]] for i in consider_top]
        class_of_sample = GT_samples[given_sample]
        T = top_sample_classes.count(class_of_sample)
        R = top_k
        sum_term = 0
        for i in range(0,R):
                if top_sample_classes[i] != class_of_sample:
                        pass
                else:
                        p_r = top_sample_classes[:i+1].count(class_of_sample)
                        sum_term = sum_term + (p_r*1.0)/len(top_sample_classes[:i+1])
        if T == 0:
                return 0
        else:
                return float(sum_term/T)        

### Reading data

In [8]:
# Read the required Ground truth for the task - IMAGES
keys = os.path.join(datar,'datasets/wikipedia/testset_txt_img_cat.list')
im_txt_pair_wd = open(keys, 'r').readlines()
GT_samples = {} # In general
for i in im_txt_pair_wd:
    GT_samples[i.split('\t')[1]] = i.split('\t')[2].replace('\n','') # (Corresponding image, class)  
print(len(GT_samples))

693


In [4]:
import pickle
# reading resnet101 features 
data_f = os.path.join(datar,'datasets/wikipedia/gen','resnet_all.pkl')
with open(data_f,'rb') as f:
    images = pickle.load(f)

images_f = {}
for img_id,cl in GT_samples.items():
    try:        
        images_f[img_id] = images.get(img_id).squeeze()
    except Exception as e:
        print ('CORRUPTED IMAGE: ',img_id)
        print('Exception: %s',"".join(traceback.format_exception(*sys.exc_info())))
len(images_f)

693

In [5]:
# Read the required Ground truth for the task - TEXT
keys = os.path.join(datar,'datasets/wikipedia/testset_txt_img_cat.list')
im_txt_pair_wd = open(keys, 'r').readlines()
GT_samples = {} # In general
for i in im_txt_pair_wd:
    GT_samples[i.split('\t')[0]] = i.split('\t')[2].replace('\n','') # (Corresponding text, class)  
print(len(GT_samples))

693


In [6]:
import pickle
# reading resnet101 features
data_f = os.path.join(datar,'datasets/wikipedia/gen/test_bert.pkl')
with open(data_f,'rb') as f:
    texts = pickle.load(f)
len(texts)

texts_f = {}
for s_id,cl in GT_samples.items():
    try:        
        texts_f[s_id] = texts.get(s_id)[0]
    except Exception as e:
        print ('CORRUPTED Text: ',s_id)
        print('Exception: %s',"".join(traceback.format_exception(*sys.exc_info())))
len(texts_f)

693

### Retrieval

In [9]:
keys = os.path.join(datar,'datasets/wikipedia/testset_txt_img_cat.list')
im_txt_pair_wd = open(keys, 'r').readlines()
GT_samples = {} # In general
for i in im_txt_pair_wd:
    GT_samples[i.split('\t')[1]] = i.split('\t')[2].replace('\n','') # (Corresponding image, class)  
    GT_samples[i.split('\t')[0]] = i.split('\t')[2].replace('\n','') # (Corresponding text, class)  
print(len(GT_samples))

1386


In [11]:
import sys
sys.path.append('/multimodal_retrieval/')
from evaluation import eval_wiki_retrieval2 as eval_r

distances = ['corr','eu','cos']
topks = [8,50,500,692]  

maps1 = eval_r.calc_distances(images_f,images_f,'img2img',topks,distances,GT_samples)
maps2 = eval_r.calc_distances(texts_f,texts_f,'txt2txt',topks,distances,GT_samples)
mapt_uni = pd.concat([maps1, maps2], axis=0)
mapt_uni = mapt_uni[['task','distances']+topks]
mapt_uni

,task,distances,8,50,500,692
0,img2img,corr,0.448019,0.331130,0.205979,0.195581
1,img2img,eu,0.433935,0.323024,0.193828,0.180504
2,img2img,cos,0.140580,0.130297,0.092877,0.098422
0,txt2txt,corr,0.732741,0.594859,0.402484,0.382778
1,txt2txt,eu,0.735880,0.600428,0.406017,0.387331
2,txt2txt,cos,0.068883,0.059512,0.059135,0.078817
